In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Configuración
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.cardmarket.com/es/Magic/Users/UrbanCardsMTG/Offers/Singles"
driver.get(url)

print("Esperando que la página cargue...")
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# Aceptar cookies
try:
    print("Esperando el botón de cookies...")
    accept_btn = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar todas las cookies')]"))
    )
    accept_btn.click()
    print("Botón de cookies aceptado.")
except Exception as e:
    print("No se pudo aceptar cookies:", e)

# Esperar las cartas
try:
    print("Esperando las cartas...")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.article-row"))
    )
    offers = driver.find_elements(By.CSS_SELECTOR, "div.article-row")
    print(f"Se encontraron {len(offers)} cartas.")
except Exception as e:
    print("No se encontraron cartas:", e)
    driver.quit()
    exit()

# Extraer la información
cards = []
for offer in offers:
    try:
        name = offer.find_element(By.CSS_SELECTOR, "a[href*='/es/Magic/Products/Singles/']").text.strip()
    except:
        name = "N/A"
    
    try:
        edition = offer.find_element(By.CSS_SELECTOR, "a.expansion-symbol").get_attribute("aria-label").strip()
    except:
        edition = "N/A"

    try:
        condition = offer.find_element(By.CSS_SELECTOR, "a.article-condition span.badge").text.strip()
    except:
        try:
            # Otra forma común de mostrar la condición
            condition = offer.find_element(By.XPATH, ".//div[contains(@class, 'product-availability-details')]//span").text.strip()
        except:
            condition = "N/A"

    try:
        language = offer.find_element(By.CSS_SELECTOR, "span.icon[aria-label]").get_attribute("aria-label").strip()
    except:
        language = "N/A"

    try:
        price = offer.find_element(By.CSS_SELECTOR, "div.price-container").text.strip().replace("€", "").replace(",", ".")
        price = float(price)
    except:
        price = -1

    try:
        # Usar el nuevo selector para la cantidad
        quantity_element = offer.find_element(By.CSS_SELECTOR, "div.amount-container.d-none.d-md-flex.justify-content-end.me-3 span.item-count")
        
        # Depuración: Imprimir el contenido de quantity_element para verificar que estamos extrayendo el valor correcto
        quantity_text = quantity_element.text.strip()
        print(f"Cantidad encontrada: {quantity_text}")  # Depuración para ver el valor de cantidad

        # Verificar que la cantidad existe y no está vacía
        if quantity_text and quantity_text.isdigit():  # Asegurarnos de que la cantidad es un número
            quantity = int(quantity_text)
        else:
            quantity = -1  # Si está vacío, asignamos -1
    except Exception as e:
        quantity = -1
        print(f"Error extrayendo cantidad: {e}")

    cards.append({
        "name": name,
        "edition": edition,
        "condition": condition,
        "language": language,
        "price_eur": price,
        "quantity": quantity
    })

driver.quit()

# Guardar CSV
df = pd.DataFrame(cards)
df.to_csv("urban_cards_inventory_page1.csv", index=False, encoding="utf-8-sig")
print(f"Scraping completado. Se extrajeron {len(df)} cartas.")


Esperando que la página cargue...
Esperando el botón de cookies...
Botón de cookies aceptado.
Esperando las cartas...
Se encontraron 20 cartas.
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Scraping completado. Se extrajeron 20 cartas.
